In [ ]:
from collections import defaultdict
from datetime import datetime
import glob

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
ylims = {
    'tan_phi': [0.63, 0.78],
    'compression_factor': [0, 12000],
    'C_lab': [0.5e6, 2.2e6],
    'ERA5_quad_drag_coef_air': [0.0013, 0.0035],
}

label_name_map = {
    'compression_factor': '$P_0$',
    'C_lab': '$c_\\text{ref}$',
    'exponent_compression_factor': '$H$',
    'compaction_param': '$C$',
    'ERA5_quad_drag_coef_air': '$C_A$',
    'nu0': '$\\nu$',
    'tan_phi': '$\\mu$',
}

plot_configs = [
    dict(
        param_names = ['tan_phi'],
        configs = ['cfg01'],
    ),
    dict(
        param_names = ['compression_factor', 'C_lab'],
        configs = ['cfg01', 'cfg02'],
    ),
    dict(
        param_names = ['ERA5_quad_drag_coef_air'],
        configs = ['cfg02'],
    ),
]

In [ ]:
for plot_config in plot_configs:
    param_names = plot_config['param_names']
    configs = plot_config['configs']
    stack = defaultdict(list)
    for config in configs:
        ifiles = sorted(glob.glob(f'../music_matrix/rgps/*inference_{config}.npz'))
        for ifile in ifiles:
            with np.load(ifile) as f:
                d = dict(**f)
            stack[f'date_{config}'].append(np.datetime64(d['date'][-1], 'Y').item().year)
            for param_name in param_names:
                p_avg = np.mean(d[param_name])
                p_std = np.std(d[param_name])
                stack[f'{param_name}_{config}_avg'].append(p_avg)
                stack[f'{param_name}_{config}_std'].append(p_std)
    for i in stack:
        stack[i] = np.hstack(stack[i])

    for config in configs:
        srt_idx = np.argsort(stack[f'date_{config}'])
        stack[f'date_{config}'] = stack[f'date_{config}'][srt_idx]
        for param_name in param_names:
            for stat_name in ['avg', 'std']:
                i = f'{param_name}_{config}_{stat_name}'
                stack[i] = stack[i][srt_idx]

    for param_name in param_names:
        fig = plt.figure(figsize=(4,3))
        for config in configs:
            date = stack[f'date_{config}']
            avg = stack[f'{param_name}_{config}_avg']
            std = stack[f'{param_name}_{config}_std']
            l = plt.plot(date, avg, '.-', label=config.replace('cfg0', ''))
            plt.fill_between(date, avg-std, avg+std, alpha=0.5, color=l[0].get_color())
        ofile = f'../tuning_paper_figures/fig00_{param_name}_annual.png'
        plt.ylim(ylims[param_name])
        plt.savefig(ofile, dpi=100, bbox_inches='tight', pad_inches=0.1)
        plt.close()
        print(ofile)


In [ ]:
for plot_config in plot_configs:
    param_names = plot_config['param_names']
    configs = plot_config['configs']
    stack = defaultdict(list)
    for config in configs:
        ifiles = sorted(glob.glob(f'../music_matrix/rgps/*inference_{config}.npz'))
        for ifile in ifiles:
            with np.load(ifile) as f:
                d = dict(**f)
            months = np.array([i.item().month for i in d['date'].astype(np.datetime64(1, 'D'))])
            months[months == 12] = 0
            stack[f'date_{config}'].append(months)
            for param_name in param_names:
                stack[f'{param_name}_{config}'].append(d[param_name])
    for i in stack:
        stack[i] = np.hstack(stack[i])

    for param_name in param_names:
        monthly = defaultdict(list)
        for config in configs:
            unq_dates = np.unique(stack[f'date_{config}'])
            for unq_date in unq_dates:
                unq_date_indx = stack[f'date_{config}'] == unq_date
                unq_date_data = stack[f'{param_name}_{config}'][:, unq_date_indx]
                monthly[f'month_{config}'].append(unq_date)
                monthly[f'{param_name}_{config}_avg'].append(unq_date_data.mean())
                monthly[f'{param_name}_{config}_std'].append(unq_date_data.std())
        for i in monthly:
            monthly[i] = np.hstack(monthly[i])

        fig = plt.figure(figsize=(4,3))
        for config in configs:
            avg = monthly[f'{param_name}_{config}_avg']
            std = monthly[f'{param_name}_{config}_std']
            l = plt.plot(monthly[f'month_{config}'], avg, '.-', label=config)
            l = plt.fill_between(monthly[f'month_{config}'], avg-std, avg+std, alpha=0.5, color=l[0].get_color())
        #plt.title(param_name)
        plt.xticks([0, 1, 2, 3, 4, 5], ['Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May'])
        #plt.show()
        ofile = f'../tuning_paper_figures/fig00_{param_name}_monthly.png'
        plt.ylim(ylims[param_name])
        plt.savefig(ofile, dpi=100, bbox_inches='tight', pad_inches=0.1)
        plt.close()
        print(ofile)


In [ ]:
for plot_config in plot_configs:
    param_names = plot_config['param_names']
    configs = plot_config['configs']
    for param_name in param_names:
        fig = plt.figure(figsize=(4,3))
        for config in configs:
            ifile = f'../music_matrix/rgps/w07_may_inference_{config}.npz'
            with np.load(ifile) as f:
                d = dict(**f)
            avg = d[param_name].mean(axis=0)
            std = d[param_name].std(axis=0)
            l = plt.plot(d['date'], avg, '.-', label=config.replace('cfg0', ''))
            plt.fill_between(d['date'], avg-std, avg+std, color=l[0].get_color(), alpha=0.5)
        plt.legend()
        #plt.title(param_name)
        plt.xticks(rotation=15)
        #plt.show()
        ofile = f'../tuning_paper_figures/fig00_{param_name}_daily.png'
        plt.ylim(ylims[param_name])
        plt.ylabel(label_name_map[param_name])
        plt.savefig(ofile, dpi=100, bbox_inches='tight', pad_inches=0.1)
        plt.close()
        print(ofile)
